# Modelo genérico



## Índices / Conjuntos

$I \colon \{1,\ldots, m\}$

$J \colon \{1,\ldots, n\}$

## Parâmetros

$d_{ij} \colon  \text{distância do ponto } i \in I \text{ até o ponto } j \in J$

$p = \text{número de medianas}$


## Variáveis de decisão

$
    y_{j}=
    \begin{cases}
      1, & \text{se o ponto } j \in J \text{ é escolhido como mediana} \\
      0, & \text{caso contrário}
    \end{cases}
$

$
    x_{ij}=
    \begin{cases}
      1, & \text{se o ponto } i \in I \text{ é alocado à mediana } j \in J \\
      0, & \text{caso contrário}
    \end{cases}
$

## Formulação matemática

$\text{min }z = \sum\limits_{i \in I} \sum\limits_{j \in J} d_{ij} x_{ij}$

sujeito a

$\sum\limits_{j \in J} y_{j} = p$

$\sum\limits_{j \in J} x_{ij} = 1, \;\; \forall i \in I$

$x_{ij} \leq y_j, \;\; \forall i \in I,j \in J$

$y_j \in \{0,1\}, \;\; \forall \; j \in J$

$x_{ij} \in \{0,1\}, \;\; \forall i \in I, j \in J$

In [1]:
# Carrega as bibliotecas necessárias:

import pyomo.environ as pyo

In [2]:
# Declara o modelo computacional:

modelo = pyo.AbstractModel()

# Parâmetros auxiliares:
modelo.m = pyo.Param()
modelo.n = pyo.Param()
modelo.distancias = pyo.Param(within=pyo.Any)

# Índices:
modelo.I = pyo.RangeSet(modelo.m)
modelo.J = pyo.RangeSet(modelo.n)

# Parâmetros:
modelo.d = pyo.Param(modelo.I, modelo.J, initialize=lambda modelo, i, j: modelo.distancias()[i-1][j-1])
modelo.p = pyo.Param()

# Variáveis de decisão:
modelo.y = pyo.Var(modelo.J, within=pyo.Binary)
modelo.x = pyo.Var(modelo.I, modelo.J, within=pyo.Binary)

# Função objetivo:
def f_obj(modelo):
    return sum(modelo.x[i,j] * modelo.d[i,j] for i in modelo.I for j in modelo.J)
modelo.z = pyo.Objective(rule=f_obj, sense=pyo.minimize)

# Restrições:

def f_restr1(modelo):
    return sum(modelo.y[j] for j in modelo.J) == modelo.p()
modelo.restr_1 = pyo.Constraint(rule=f_restr1)

def f_restr2(modelo, i):
    return sum(modelo.x[i,j] for j in modelo.J) == 1
modelo.restr_2 = pyo.Constraint(modelo.I, rule=f_restr2)

def f_restr3(modelo, i, j):
    return modelo.x[i,j] <= modelo.y[j]
modelo.restr_3 = pyo.Constraint(modelo.I, modelo.J, rule=f_restr3)

In [3]:
# Lê os dados de entrada:

dados_entrada = pyo.DataPortal()

dados_entrada.load(filename='dados_entrada.json')

In [4]:
# Cria a instância:

instancia = modelo.create_instance(dados_entrada)

In [5]:
instancia.pprint()

3 Set Declarations
    d_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    I*J :   25 : {(1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5)}
    restr_3_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    I*J :   25 : {(1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5)}
    x_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    I*J :   25 : {(1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (5, 1), (5, 2

In [6]:
# Resolve a instância:

resultado = pyo.SolverFactory('glpk').solve(instancia)
print(resultado)

    '['/Users/acsjunior/opt/anaconda3/envs/ppgmne-prf/bin/glpsol', '--
    version']' timed out after 1 seconds
Traceback (most recent call last):
  File "/Users/acsjunior/opt/anaconda3/envs/ppgmne-prf/lib/python3.8/site-packages/pyomo/opt/base/solvers.py", line 152, in __call__
    opt = self._cls[_name](**kwds)
  File "/Users/acsjunior/opt/anaconda3/envs/ppgmne-prf/lib/python3.8/site-packages/pyomo/solvers/plugins/solvers/GLPK.py", line 64, in __new__
    configure_glpk()
  File "/Users/acsjunior/opt/anaconda3/envs/ppgmne-prf/lib/python3.8/site-packages/pyomo/solvers/plugins/solvers/GLPK.py", line 35, in configure_glpk
    result = subprocess.run([Executable('glpsol').path(), "--version"],
  File "/Users/acsjunior/opt/anaconda3/envs/ppgmne-prf/lib/python3.8/subprocess.py", line 495, in run
    stdout, stderr = process.communicate(input, timeout=timeout)
  File "/Users/acsjunior/opt/anaconda3/envs/ppgmne-prf/lib/python3.8/subprocess.py", line 1028, in communicate
    stdout, stderr = 

RuntimeError: Attempting to use an unavailable solver.

The SolverFactory was unable to create the solver "glpk"
and returned an UnknownSolver object.  This error is raised at the point
where the UnknownSolver object was used as if it were valid (by calling
method "solve").

The original solver was created with the following parameters:
	type: glpk
	_args: ()
	options: {}

In [ ]:
instancia.y.pprint()

In [7]:
instancia.x.pprint()

x : Size=25, Index=x_index
    Key    : Lower : Value : Upper : Fixed : Stale : Domain
    (1, 1) :     0 :  None :     1 : False :  True : Binary
    (1, 2) :     0 :  None :     1 : False :  True : Binary
    (1, 3) :     0 :  None :     1 : False :  True : Binary
    (1, 4) :     0 :  None :     1 : False :  True : Binary
    (1, 5) :     0 :  None :     1 : False :  True : Binary
    (2, 1) :     0 :  None :     1 : False :  True : Binary
    (2, 2) :     0 :  None :     1 : False :  True : Binary
    (2, 3) :     0 :  None :     1 : False :  True : Binary
    (2, 4) :     0 :  None :     1 : False :  True : Binary
    (2, 5) :     0 :  None :     1 : False :  True : Binary
    (3, 1) :     0 :  None :     1 : False :  True : Binary
    (3, 2) :     0 :  None :     1 : False :  True : Binary
    (3, 3) :     0 :  None :     1 : False :  True : Binary
    (3, 4) :     0 :  None :     1 : False :  True : Binary
    (3, 5) :     0 :  None :     1 : False :  True : Binary
    (4, 1) : 